#Import Dataframes and Instantiate Cerebro Backtrader Engine

In [78]:
# import pandas as pd
import backtrader as bt
from backtrader import cerebro
import backtrader.feeds as btfeeds

os.getcwd()
os.chdir(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data')

# df_stock = pd.read_csv(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPL.csv')
# df_fin = pd.read_csv(r"C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPLfinancials.csv")

cerebro = bt.Cerebro()

pricedf = bt.feeds.YahooFinanceCSVData(dataname='AAPL.csv', reverse=False)
ratiodf = bt.feeds.YahooFinanceCSVData(dataname='AAPLfinancials.csv', reverse=False)

cerebro.adddata(pricedf, ratiodf)
print(pricedf, ratiodf) #why are the df's not printing?

cerebro.broker.set_cash(2500)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

<backtrader.feeds.yahoo.YahooFinanceCSVData object at 0x000002C58FAE76A0> <backtrader.feeds.yahoo.YahooFinanceCSVData object at 0x000002C58FB29490>
Starting Portfolio Value: 2500.00
Final Portfolio Value: 2500.00


#Create MACD Strategy Class

In [93]:
import backtrader.indicators as btind

class MACD_Crossover(bt.Strategy):
    """
    This class serves as template for developing trading strategies using the MACD
    technical indicator. The buy logic is dependant on a crossover that occurs with
    the MACD indicator.
    """
   
    def log(self, txt, dt=None):
        ''' Logging function for strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.macd = bt.indicators.MACD(self.data,
                                        period_me1=self.p.macd1,
                                        period_me2=self.p.macd2,
                                        period_signal=self.p.macdsig)
 
        # Cross of macd.macd and macd.signal
        self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

        set stop price indicator
        self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)
        
        self.MACD = btind.MACD(period=9)
        self.CrossOver = btind.CrossOver
        # self.CrossOver = self.MACD
        # self.Crossover(data = self.MACD)

    def next(self):
        if self.CrossOver == 1:
            # buy stock
            pass
        elif self.CrossOver == -1:
            # sell stock
            pass
        # else:
            pass

macd1 = MACD_Crossover()

        
    


AttributeError: 'NoneType' object has no attribute '_next_stid'

#Create SMA Crossover 1 Strategy Class

#Create SMA Crossover 2 Strategy Class

#Create Mean Reversion Strategy